# Peer-graded assignment 2: Segmenting and Clustering Neighborhoods in Toronto


## Table of Contents

<font size = 3>

1. [Downloading and Exploring Dataset](#item1)
2. [Segmenting Neighborhoods in Toronto boroughs with the word 'Toronto'](#item2)
3. [Clustering Neighborhoods in Toronto boroughs with the word 'Toronto'](#item3)

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a class="anchor" id="item1"></a> 

## 1. Download and Explore Dataset 

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

toronto_data = pd.read_html(wiki_url)

In [3]:
df_toronto = toronto_data[0]
print(df_toronto.shape[0])
df_toronto.head()

180


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:


df1 = pd.DataFrame(df_toronto[df_toronto[['Postal Code', 'Borough','Neighbourhood']] == 'Not assigned'].count())
df1.rename(columns = {0:'Not assigned'}, inplace = True)

df2 = pd.DataFrame(df_toronto[df_toronto[['Postal Code', 'Borough','Neighbourhood']] != 'Not assigned'].count())
df2.rename(columns = {0:'Assigned'}, inplace = True)

df_class = pd.concat([df1,df2], axis = 1).transpose()

df_class

,Postal Code,Borough,Neighbourhood
Not assigned,0,77,77
Assigned,180,103,103


In [5]:
df_neigbhourhood = df_toronto[df_toronto['Neighbourhood'] != 'Not assigned'].reset_index(drop = True)

print('Toronto, Ontario has {} boroughs and {} neighborhoods.'.format(
                                                                    len(df_neigbhourhood['Borough'].unique()),
                                                                    df_neigbhourhood.shape[0]
                                                                    )
     )

df_neigbhourhood.head(12)

Toronto, Ontario has 10 boroughs and 103 neighborhoods.


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
# !pip install geocoder

import geocoder # import geocoder
postal_code = df_neigbhourhood['Postal Code']

# Using while Statement
latitude = []
longitude = []
n = 0

while n < len(postal_code):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code[n]))
    lat_lng_coords = g.latlng
    #print('The geograpical coordinate of {} are {}, {}.'.format(postal_code[n], lat_lng_coords[0], lat_lng_coords[1]))
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    n = n + 1

# OR Using for Statement
# latitude = []
# longitude= []

# for p_code in postal_code:
#     g = geocoder.arcgis('{}, Toronto, Ontario'.format(p_code))
#     lat_lng_coords = g.latlng
#     #print('The geograpical coordinate of {} are {}, {}.'.format(p_code, lat_lng_coords[0], lat_lng_coords[1]))
#     latitude.append(lat_lng_coords[0])
#     longitude.append(lat_lng_coords[1])

In [7]:
print ('Using while-statement: The geograpical coordinates of Postal Code {} are latitude {}, longitude {}.'.format(
                                                                                                                    np.array(postal_code[0:2]), 
                                                                                                                    np.array(latitude[0:2]).round(4), 
                                                                                                                    np.array(longitude[0:2]).round(4)
                                                                                                                    )
      )

Using while-statement: The geograpical coordinates of Postal Code ['M3A' 'M4A'] are latitude [43.7525 43.7306], longitude [-79.3299 -79.3131].


In [8]:
df_neigbhourhood['Latitude'] = latitude
df_neigbhourhood['Longitude'] = longitude

df_neigbhourhood.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66263,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,M3B,North York,Don Mills,43.74923,-79.36186
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


<a class="anchor" id="item2"></a> 

## 2. Segmenting Neighborhoods in Toronto boroughs with the word 'Toronto

In [9]:
import os

from dotenv import load_dotenv
load_dotenv() 

CLIENT_ID = os.getenv('Foursquare_ID')               # your Foursquare ID
CLIENT_SECRET = os.getenv('Foursquare_Secret')       # your Foursquare Secret
ACCESS_TOKEN = os.getenv('Foursquare_AccessToken')   # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

print('Credentials: successful')

Credentials: successful


### Visualizing Toronto neighbourhoods on a map ###

In [10]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_neigbhourhood['Latitude'], df_neigbhourhood['Longitude'], df_neigbhourhood['Borough'], df_neigbhourhood['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Exploring and Visualizing Boroughs with the word 'Toronto' on a map ###

In [12]:
TO_borough = df_neigbhourhood[df_neigbhourhood['Borough'].str.contains("Toronto")].reset_index(drop=True)

print('Total neighbourhoods in boroughs with the word \'Toronto\': ' , TO_borough.shape[0])
TO_borough.head()

Total neighbourhoods in boroughs with the word 'Toronto':  39


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,M4E,East Toronto,The Beaches,43.67709,-79.29547


In [13]:
map_borough = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(TO_borough['Latitude'], TO_borough['Longitude'], TO_borough['Borough'], TO_borough['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_borough)  
    
map_borough

### Exploring first neighbourhood (Regent Park, Harbourfront) and discovering the nearby venues within 500 meters ###

In [14]:
neighbourhood_latitude = TO_borough.loc[0, 'Latitude']   
neighbourhood_longitude = TO_borough.loc[0, 'Longitude'] 

neighbourhood_name = TO_borough.loc[0, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.65512000000007, -79.36263999999994.


In [15]:
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format (CLIENT_ID,
                                                                                                                            CLIENT_SECRET,
                                                                                                                            VERSION,
                                                                                                                            neighbourhood_latitude, 
                                                                                                                            neighbourhood_longitude,                                                                                                                                           
                                                                                                                            radius, 
                                                                                                                            LIMIT
                                                                                                                            )

print('URL defined')

URL defined


In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ffc23471b871d59f8aeed8d'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.65962000450007,
    'lng': -79.3564319112327},
   'sw': {'lat': 43.650619995500065, 'lng': -79.36884808876718}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance':

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

22 venues were returned by Foursquare.


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,The Yoga Lounge,Yoga Studio,43.655515,-79.364955


### Exploring all neighbourhoods in the boroughs with the word 'Toronto' and discovering the nearby venues within 500 meters ###

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                             'Neighbourhood Latitude', 
                             'Neighbourhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)


In [20]:
TO_borough_venues = getNearbyVenues(names=TO_borough['Neighbourhood'],
                                   latitudes=TO_borough['Latitude'],
                                   longitudes=TO_borough['Longitude']
                                   )

In [21]:
print('Total nearby venues in boroughs with the word \'Toronto\': ' , TO_borough_venues.shape[0])
TO_borough_venues.head()

Total nearby venues in boroughs with the word 'Toronto':  813


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


In [22]:
TO_borough_venues.groupby('Neighbourhood').count().head(10)

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",30,30,30,30,30,30
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",30,30,30,30,30,30
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
Christie,11,11,11,11,11,11
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,28,28,28,28,28,28


<a class="anchor" id="item3"></a> 

## 3. Clustering Neighborhoods in Toronto boroughs with the word 'Toronto

In [23]:
TO_borough_onehot = pd.get_dummies(TO_borough_venues[['Venue Category']], prefix="", prefix_sep="")


TO_borough_onehot['Neighbourhood'] = TO_borough_venues['Neighbourhood'] 


fixed_columns = [TO_borough_onehot.columns[-1]] + list(TO_borough_onehot.columns[:-1])
TO_borough_onehot = TO_borough_onehot[fixed_columns]

print(TO_borough_onehot.shape)
TO_borough_onehot.head()

(813, 180)


,Neighbourhood,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Opera House,Organic Grocery,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [24]:
TO_borough_grouped = TO_borough_onehot.groupby('Neighbourhood').mean().reset_index()

print(TO_borough_grouped.shape)
TO_borough_grouped.head()

(39, 180)


,Neighbourhood,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Opera House,Organic Grocery,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.033333,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.033333,0.0,0.000000,0.0,0.066667,0.0,0.0,0.033333,0.0,0.000000,0.0,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.066667,0.000000,0.066667,0.066667,0.0,0.0,0.033333,0.000000,0.033333,0.0,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.066667,0.000000,0.0,0.033333,0.0,0.0,0.000000,0.0,0.000000,0.033333,0.033333,0.033333,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.033333,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.033333,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.000000,0.000000,0.033333,0.000000,0.0,0.0,0.0,0.066667,0.0,0.033333,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.033333,0.0,0.0,0.000000,0.000000,0.000000,0.033333,0.066667,0.0,0.0,0.000000,0.000000,0.00

In [25]:
num_top_venues = 5

for hood in TO_borough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = TO_borough_grouped[TO_borough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
   

----Berczy Park----
                venue  freq
0         Cheese Shop  0.07
1  Seafood Restaurant  0.07
2        Cocktail Bar  0.07
3         Coffee Shop  0.07
4      Farmers Market  0.07


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0               Gift Shop  0.07
1             Coffee Shop  0.07
2      Italian Restaurant  0.07
3                  Bakery  0.07
4  Furniture / Home Store  0.07


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
          venue  freq
0          Café  0.10
1         Hotel  0.07
2   Pizza Place  0.07
3       Theater  0.07
4  Concert Hall  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0    Italian Restaurant  0.10
1  Gym / Fitness Center  0.07
2            Restaurant  0.07
3                  Park  0.07
4           Men's Store  0.03


----Central Bay Street----
            

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = TO_borough_grouped['Neighbourhood']

for ind in np.arange(TO_borough_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TO_borough_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Cheese Shop,Seafood Restaurant,Cocktail Bar,Coffee Shop,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Gift Shop,Coffee Shop,Italian Restaurant,Bakery,Furniture / Home Store
2,"Business reply mail Processing Centre, South C...",Café,Hotel,Pizza Place,Theater,Concert Hall
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Gym / Fitness Center,Restaurant,Park,Men's Store
4,Central Bay Street,Coffee Shop,Plaza,Clothing Store,Ramen Restaurant,Bubble Tea Shop


In [28]:
kclusters = 5

TO_borough_clustering = TO_borough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TO_borough_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 0, 1, 1, 4, 1])

In [29]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

TO_borough_merged = TO_borough

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
TO_borough_merged = TO_borough_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

TO_borough_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,1,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Wine Shop,Theater
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,1,Coffee Shop,Sandwich Place,Park,Burrito Place,Café
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,1,Café,Ramen Restaurant,Coffee Shop,Theater,Tanning Salon
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,1,Café,Gastropub,Coffee Shop,Cosmetics Shop,Theater
4,M4E,East Toronto,The Beaches,43.67709,-79.29547,0,Health Food Store,Pub,Trail,Neighborhood,Monument / Landmark


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TO_borough_merged['Latitude'], TO_borough_merged['Longitude'], TO_borough_merged['Neighbourhood'], TO_borough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the Clusters ###

Cluster 1

In [31]:
TO_borough_merged.loc[TO_borough_merged['Cluster Labels'] == 0, TO_borough_merged.columns[[1,2] + list(range(5, TO_borough_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,East Toronto,The Beaches,0,Health Food Store,Pub,Trail,Neighborhood,Monument / Landmark
22,West Toronto,"High Park, The Junction South",0,Residential Building (Apartment / Condo),Convenience Store,Park,Movie Theater,Massage Studio


Cluster 2

In [32]:
TO_borough_merged.loc[TO_borough_merged['Cluster Labels'] == 1, TO_borough_merged.columns[[1,2] + list(range(5, TO_borough_merged.shape[1]))]]


,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",1,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Wine Shop,Theater
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Sandwich Place,Park,Burrito Place,Café
2,Downtown Toronto,"Garden District, Ryerson",1,Café,Ramen Restaurant,Coffee Shop,Theater,Tanning Salon
3,Downtown Toronto,St. James Town,1,Café,Gastropub,Coffee Shop,Cosmetics Shop,Theater
5,Downtown Toronto,Berczy Park,1,Cheese Shop,Seafood Restaurant,Cocktail Bar,Coffee Shop,Farmers Market
6,Downtown Toronto,Central Bay Street,1,Coffee Shop,Plaza,Clothing Store,Ramen Restaurant,Bubble Tea Shop
7,Downtown Toronto,Christie,1,Café,Grocery Store,Playground,Candy Store,Athletics & Sports
8,Downtown Toronto,"Richmond, Adelaide, King",1,Café,Coffee Shop,Seafood Restaurant,Gym,Restaurant
9,West Toronto,"Dufferin, Dovercourt Village",1,Furniture / Home Store,Park,Pet Store,Bakery,Café
10,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",1,Hotel,Coffee Shop,Park,Japanese Restaurant,Aquarium


Cluster 3

In [33]:
TO_borough_merged.loc[TO_borough_merged['Cluster Labels'] == 2, TO_borough_merged.columns[[1,2] + list(range(5, TO_borough_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,Central Toronto,Roselawn,2,Home Service,American Restaurant,Movie Theater,Massage Studio,Mediterranean Restaurant


Cluster 4

In [34]:
TO_borough_merged.loc[TO_borough_merged['Cluster Labels'] == 3, TO_borough_merged.columns[[1,2] + list(range(5, TO_borough_merged.shape[1]))]].head()

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
23,Central Toronto,"North Toronto West, Lawrence Park",3,Gym Pool,Playground,Park,Movie Theater,Massage Studio
33,Downtown Toronto,Rosedale,3,Park,Playground,Bike Trail,American Restaurant,Monument / Landmark


Cluster 5

In [35]:
TO_borough_merged.loc[TO_borough_merged['Cluster Labels'] == 4, TO_borough_merged.columns[[1,2] + list(range(5, TO_borough_merged.shape[1]))]].head()

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,East Toronto,"The Danforth West, Riverdale",4,Bus Line,Park,Grocery Store,Business Service,Ice Cream Shop
18,Central Toronto,Lawrence Park,4,Bus Line,Business Service,Swim School,American Restaurant,Monument / Landmark
21,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",4,Park,Business Service,American Restaurant,Monument / Landmark,Massage Studio
